Credit: 

https://github.com/aimacode/aima-python

**NOTE**: 

If you are using google colab, you need to assign the path of present working directory (in Google Drive) to path.

     path = '/content/drive/My Drive/xxxx/xxxxxx' 

In [48]:
# We need the following code block if we are importing modules or working with data files from current working directiory

# For Google Colab
# Upload the folder containing this file to google drive.
# Make sure that uninformed_informed_search.py is also in the same folder in google drive.
import sys, os

# Checking if the notebook is opened in google colab
#If YES, mount the google drive and change the directory
if 'google.colab' in sys.modules:

    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    # change path to the folder 
    path = '/content/drive/My Drive/IT5005/Lect1_Uninf_Inf_Search'
    print(path)
    #os.chdir changes the current working directory
    os.chdir(path)
    !pwd


In [49]:
if 'google.colab' in sys.modules:
    # check the contents of present working directory
    print(os.listdir(path))

In [50]:
from uninformed_informed_search import *

 8-puzzle


![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a goal configuration. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numebred 1 to 8, with 0 representing the blank. An action is the cell index number that is to be swapped with the blank (*not* the actual number to be swapped but the index into the state). So the diagram above left is the state `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, and the action is `8`, because the cell number 8 (the 9th or last cell, the `6` in the bottom right) is swapped with the blank.

There are two disjoint sets of states that cannot be reached from each other. One set has an even number of "inversions"; the other has an odd number. An inversion is when a piece in the state is larger than a piece that follows it.



In [51]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal

    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]

    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)

    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)


# hamming distance is equal to number of miplaced tiles
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))


def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))


def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height)
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

## Inversion check

It is used to find whether the slide puzzle is solvable or not.

Inversion occurs when two tiles are in wrong order

Only if the number of inversions are even, then the puzzle is solvable

### Example 1:

state = (4,2, 3, 1, 0, 5, 6, 7, 8)

Inversions: (4,2), (4,3), (4,1), (2,1), and (3,1)

Number of inversions are 5 (odd)

### Example 2:

state = (4, 2, 1, 3, 0, 5, 6, 7, 8)

Inversions: (4,2), (4,1), (4,3), and (2,1)

Number of inversions are 4 (even)



In [52]:
#Inversion check is used to find whether the slide puzzle is solvable or not.
#Inversion occurs when two tiles are in wrong order
state = (4,2, 1, 3, 0, 5, 6, 7, 8)
#Inversions: (4,2), (4,3), (4,1), (2,1), and (3,1)
inversions(state)

4

# Heuristics for 8 Puzzle Problem

In [53]:

import math

def misplaced_tiles(node):
    goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])

def manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0

    for i in range(len(state)):
        index_state[state[i]] = index[i]

    mhd = 0

    for i in range(8):
        for j in range(2):
            mhd = abs(index_goal[i][j] - index_state[i][j]) + mhd

    return mhd

def sqrt_manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0

    for i in range(len(state)):
        index_state[state[i]] = index[i]

    mhd = 0

    for i in range(8):
        for j in range(2):
            mhd = (index_goal[i][j] - index_state[i][j])**2 + mhd

    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = misplaced_tiles(node)
    return max(score1, score2)

# 8-Puzzle Problems

In [54]:
# Some specific EightPuzzle problems

e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))
e6 = EightPuzzle((8, 7, 6, 5, 2, 4, 3, 0, 1))

In [55]:
p = e6

# Solving 8-Puzzle with Uniform-Cost Search Algorithm

In [56]:
# Solve an 8 puzzle problem with uniform cost search and print out each state
result_ucs = uniform_cost_search(p)
print(f'Number of movements: {result_ucs.path_cost}')
'''
for s in path_states(result_ucs):
    print(board8(s))
'''

Number of movements: 27


'\nfor s in path_states(result_ucs):\n    print(board8(s))\n'

# Solving 8-Puzzle A* Search Algorithm

In [57]:
# Solve an 8 puzzle problem with astar search (misplaced tiles heuristic) and print out each state
result_astar = astar_search(p,misplaced_tiles)
print(f'Number of movements: {result_astar.path_cost}')
'''
for s in path_states(result_astar):
    print(board8(s))
'''

Number of movements: 27


'\nfor s in path_states(result_astar):\n    print(board8(s))\n'

In [58]:
# Solve an 8 puzzle problem with A* search (Manhattan heuristic) and print out each state
result_astar = astar_search(p,manhattan)
print(f'Number of movements: {result_astar.path_cost}')
'''
for s in path_states(result_astar):
    print(board8(s))
'''

Number of movements: 27


'\nfor s in path_states(result_astar):\n    print(board8(s))\n'

In [59]:
# Solve an 8 puzzle problem with GBFS search (Manhattan heuristic) and print out each state
result_gbfs = greedy_bfs(p,manhattan)
print(f'Number of movements: {result_gbfs.path_cost}')
'''
for s in path_states(result_gbfs):
    print(board8(s))
'''

Number of movements: 51


'\nfor s in path_states(result_gbfs):\n    print(board8(s))\n'

In [60]:
# Solve an 8 puzzle problem with GBFS search (Manhattan heuristic) and print out each state
result_gbfs = greedy_bfs(p,misplaced_tiles)
print(f'Number of movements: {result_gbfs.path_cost}')
'''
for s in path_states(result_gbfs):
    print(board8(s))
'''

Number of movements: 27


'\nfor s in path_states(result_gbfs):\n    print(board8(s))\n'